## 7.2 文本预处理
对于序列数据处理问题，我们在7.1节中评估了所需的统计工具和预测时面临的挑战。
这样的数据存在许多种形式，**文本**是最常见例子之一。
例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。
本节中，我们将解析文本的常见预处理步骤。
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为**词元**（如单词和字符）。
1. 建立一个词表，将拆分的词元**映射**到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。

In [1]:
import collections
import re # 提供正则表达式的支持
from d2l import torch as d2l

### 7.2.1 读取数据集
首先，我们从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数将数据集读取到由多条文本行组成的**列表**中，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。

In [4]:
""" 
d2l.DATA_HUB[]：一个列表，用于存储文本数据
d2l.DATA_URL：亚马逊的下载加速链接
"""
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a') 

""" 将时间机器数据集加载到文本行的列表中 """
def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    """ 
        1. 使用re.sub()方法将每行中的非字母字符替换为空格
        2. 使用strip()方法去除每行两端的空格 
        3. 使用lower()方法将每行转换为小写
        4. 将处理后的每行文本添加到一个列表中，并将该列表作为函数的返回值
    """
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数：{len(lines)}')
print(lines[0])
print(lines[100])

# 文本总行数：3221
the time machine by h g wells
were three dimensional representations of his four dimensioned


### 7.2.2 词元化
下面的`tokenize`函数将文本行列表（`lines`）作为输入，
列表中的每个元素是一个文本序列（如一条文本行）。
每个文本序列又被拆分成一个词元列表，*词元*（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。

In [5]:
""" 将文本行拆分为单词或字符词元 """
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型！')

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


### 7.2.3 词表
词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们构建一个字典，通常也叫做*词表*（vocabulary），用来将字符串类型的词元映射到从$0$开始的数字索引中。
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
得到的统计结果称之为*语料*（corpus）。
然后根据每个唯一词元的出现频率，为其分配一个数字索引。
很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。

In [6]:
""" 文本词表 """
class Vocab:
    """ 初始化词表对象
        tokens: 词元列表
        min_freq: 最小频率
        reserved_tokens: 保留词元列表
    """
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None: # 如果tokens为空，则将其初始化为空列表
            tokens = []
        if reserved_tokens is None: # 如果reserved_tokens为空，则将其初始化为空列表
            reserved_tokens = []
        
        # 按出现频率排序
        counter = count_corpus(tokens) # 统计词元频率。调用count_corpus函数统计tokens中每个词元的出现次数。
        self._token_freqs = sorted(counter.items(), 
                                   key=lambda x: x[1], # 按照词元频率进行排序
                                   reverse=True) # 按照降序排列

        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens # 将保留词元列表和<unk>（未知词元）组成一个新的列表，并保存在self.idx_to_token索引列表中
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)} # 构建词元到索引的映射。使用字典推导式将self.idx_to_token中的词元和对应的索引构建成一个字典，并保存在self.token_to_idx中。
        for token, freq in self._token_freqs: # 遍历词元频率列表
            if freq < min_freq: # 如果词元的频率小于最小频率，则跳出循环
                break
            if token not in self.token_to_idx: # 如果词元不在词元到索引的映射中
                self.idx_to_token.append(token) # 将词元添加到词元列表中
                self.token_to_idx[token] = len(self.idx_to_token) - 1 # 将词元和对应的索引添加到词元到索引的映射中

    """ 返回词表的大小（词元的数量） """
    def __len__(self):
        return len(self.idx_to_token)
    
    """ 根据词元获取对应的索引 """
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)): # 如果传入的tokens不是列表或元组，则返回对应词元的索引，如果词元不存在，则返回未知词元的索引。
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens] # 如果传入的tokens是列表或元组，则返回对应词元列表的索引列表。
    
    """ 根据索引获取对应的词元 """
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)): # 如果传入的indices不是列表或元组，则返回对应索引的词元。
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices] # 如果传入的indices是列表或元组，则返回对应索引列表的词元列表。
    
    """ 
        @property 是一个装饰器（decorator），用于将一个方法转换为属性访问。当我们在一个方法前面加上 @property 装饰器时，这个方法就可以像访问属性一样被调用，而不需要使用括号。
    """
    @property
    # 未知词元的索引为0
    def unk(self):
        """  
            @property 修饰的 unk 方法定义了一个名为 unk 的属性。当我们访问 unk 属性时，实际上是调用了 unk 方法并返回其结果。
            这样，我们可以通过 vocab.unk 的方式来获取未知词元的索引，而不需要使用括号调用方法。
        """
        return 0

    @property
    # 返回词元频率列表
    def token_freqs(self):
        """  
            @property 修饰的 token_freqs 方法定义了一个名为 token_freqs 的属性。
            当我们访问 token_freqs 属性时，实际上是调用了 token_freqs 方法并返回其结果。
            这样，我们可以通过 vocab.token_freqs 的方式来获取词元频率列表，而不需要使用括号调用方法。
        """
        return self._token_freqs


""" 统计词元频率 """
def count_corpus(tokens):
    # 这里的tokens是1维列表或者2维列表
    if len(tokens) == 0 or isinstance(tokens[0], list): # 检查tokens的长度是否为0或者tokens[0]是否为列表类型
        # 将词元列表展平成一个一维列表
        """  
        tokens = []
        for line in tokens:
            for token in line:
                tokens.append(token)
        """
        tokens = [token for line in tokens for token in line]
    # 使用collections.Counter()函数来计算词元的频率
    return collections.Counter(tokens) 


我们首先使用时光机器数据集作为语料库来构建词表，然后打印前几个高频词元及其索引。

In [8]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在，我们可以将每一条文本行转换成一个数字索引列表。

In [9]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


### 7.2.4 整合所有功能
在使用上述函数时，我们将所有功能打包到`load_corpus_time_machine`函数中，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；
1. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [10]:
""" 返回时光机器数据集的词元索引列表和词表 """
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

### 7.2.5 小结
* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。